### import libraries

In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
import cv2

### load the CNN model

In [2]:
model = tf.keras.models.load_model('my_model.h5')

### cascade classifier

In [48]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
vid = cv2.VideoCapture(0)
labels_dict={0:'mask',1:'no mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

### loop for all the real time frames 

In [49]:
while True:
    #check returns bool, frame returns the image as numpy array
    check, frame = vid.read()
    #Convert image to grayscale
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    #Detect multiple faces. Returns a list of faces in the current frame.
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #for each face in the frame we will find out whether the person is wearing a mask or not
    for top, left, bottom, right in faces:
        face_img=gray[left:left+bottom,top:top+bottom]
        # taking out the roi, ie, the face
        roi = cv2.resize(face_img,(256,256))
        test_image = image.img_to_array(roi)
        test_image = np.expand_dims(test_image , axis = 0)
        result = model.predict(test_image)
        
        label=result[0][0]
        if label>0.5:
            res = 1
        else:
            res = 0
        # labelling the face 
        cv2.rectangle(frame,(top,left),(top+bottom,left+right),color_dict[res],2)
        cv2.rectangle(frame,(top,left-40),(top+bottom,left),color_dict[res],-1)
        cv2.putText(frame, labels_dict[res], (top,left-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
#         if(label >0.5):
#             cv2.putText(frame, ("not wearing mask: %.2f " % (label*100)), (top,left-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
#         else:
#             cv2.putText(frame, ("wearing mask,: %.2f " % ((1-label)*100)), (top,left-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',frame)
    key=cv2.waitKey(1)
    # exit when q is pressed
    if key == ord('q'):
        break

#exiting the camera
vid.release()
cv2.destroyAllWindows()